### Vigenère cipher
    This program uses Kasiski's test to determine the key length based on the result of the trigram frequency, then uses it to calculate the Index of Coincidence to verify its accuracy. 

In [1]:
# analyze trigram frequency
def trigram(ciphertext):
    t_dict = {}

    for i in range(len(ciphertext) -2):
        for j in range(i+1, len(ciphertext) -1):
            temp = ''
            for k in range(i, i+3):
                temp += ciphertext[k]
        if temp not in t_dict.keys():
            t_dict[temp] = 1
        else:
            t_dict[temp] += 1
        temp = ''
    
    # sort the trigram by frequency
    t_dict_sorted = sorted(t_dict.items(), key=lambda x:x[1], reverse=True)
    tri_dict = []
    for i in range(len(t_dict_sorted)):
        if t_dict_sorted[i][1] > 3:
            tri_dict.append(t_dict_sorted[i])
    return tri_dict

In [2]:
ciphertext='EAPCAMSIZZSMWSGMEOMEDAWNEWXGEZRCMMUTXJYVLAPBIYBJLWMPWXVWXVJAGIAIQJRXUMXZVDMLSIBYERPAIMIAEPQSQUNHXCC\
WNWAELIPVYPIJTBRLDMFVISONXLDBYILSFTKFUEHUDCETXVLWIKYHYVDVFTAENTBFBXMFUMTTXHOXBYPGIMBWEITMFDZUEXVLDSEOPLGLBFCNVWX\
BRNWCGJLFNHXZPDVTHFWXVWEVXWSEZTAMFHIEIMCLDSEOPLGLBFCNVWXBVIMLWGJLTMLWGMSUMXSABFTAELVWVSMSLWMTOKIGUBYEPLGAMGRHFDT\
UGNXYEDVZABWSQQFLHKARICPTXLTZEIMMKHKZEGXAUQTAEPQKMIIYMSQTVIGWSCQKYHRLWMFTAIJWIEDBWSCQETXPDTKKUTPHPBKEKRAIURYAENT\
JZOESYXKRLVEMHMJBNXAIPRSGSHWGJIVEDDZSIHPGVQTAEVWPTZTRRGHKZEGXAUQTIGWLGCDEGXUPVSEIVGSCTEWMFRWLRMXGHPFWPLGXAZNLEFT\
IEDPLGXAJAGIAUBYEKIOTZVOGPQDVVPXVKDVZNMLWLWILWMKIPVRXEFNERYAIUDCCDUIACARNXMFHIEIMCSAERYLIPXAKSBRJTTRTBSFIWFTAIJH\
QKILEKDKZAEEFSQETXPDTKKUTPVTDZAMMGCVFTTFADTFGBGSALVVBELXWETAIGCTPTXWLUWIIGWSCQKYBRSRWLRMSXAINOKEFNEYEKIWAAVILGGC\
NFRFMLNBFAVYDICIAEWLPBLSJYGIPZSUIACOJOBXXDTCOPWLWIKTAISHAZGGQWCBFFFIVXKRLWSUIWISMSLGMRTBRKPVZTRMKPUZSNWWDNKHXMJI\
ZRIGMFVQETXPDTKKUTPZTZVSRMKCWKRXEDAGKHXMJQCJIGIKHUVDBGSALFCMSJHIIEMVSXVVDMSDDWBAMXZXVXSYVGBISIHPGVQTAETWGAGEVXAKM'

tri=trigram(ciphertext)
print(tri)

[('TAE', 5), ('PLG', 5), ('LWM', 4), ('ETX', 4), ('FTA', 4), ('GXA', 4), ('TAI', 4)]


In [3]:
# Using Kasiski's test to calculate the distance of two identical segements

segment = ['PLG', 'ETXP', 'GXAU']
# store the first position of each segment returned
for ele in segment:
    a_list = [i for i in range(len(ciphertext)) if ciphertext.startswith(ele, i)]
    # calculate the distance

    a_dist = []
    for i in range(1, len(a_list)):
        a_dist.append(a_list[i] - a_list[0])
    print("Segment:", ele,"| Positions:", a_list,"| Differences:", a_dist)

Segment: PLG | Positions: [200, 256, 312, 536, 550] | Differences: [56, 112, 336, 350]
Segment: ETXP | Positions: [408, 674, 891] | Differences: [266, 483]
Segment: GXAU | Positions: [361, 494] | Differences: [133]


The algorithm of the second part is based on Frideman's Index of Coincidence

In [4]:
# analyze signle letter frequency of the ciphertext
def frequency(ciphertext):
    a_dict = {}

    for i in range(len(ciphertext)):
        if ciphertext[i] not in a_dict.keys():
            a_dict[ciphertext[i]] = 1
        else:
            a_dict[ciphertext[i]] += 1

    return a_dict

In [5]:
#calcuate index of the coincidence 
def idx_coincidence(ciphertext):
    m = len(ciphertext)
    a_dict = frequency(ciphertext)    
    numerator = 0
    for key in a_dict.keys():
        numerator += a_dict[key]*(a_dict[key] -1)
    ioc = numerator /( m* (m-1))
    return ioc

In [6]:
# probability distribution of the ciphertext
def prob(ciphertext):
    a_dict = frequency(ciphertext)
    m = len(ciphertext)
    #print(a_dict)
    dictionary = list('ABCDEFGHIJKLMNOPQRSTUVWXYZ')

    prob_list = [0]*len(dictionary)
    for i in range(len(dictionary)):      
       if dictionary[i] in a_dict.keys():
           prob_list[dictionary.index(dictionary[i])] = round(a_dict[dictionary[i]] / m, 3)
       else:
           prob_list[dictionary.index(dictionary[i])] = 0       

    return prob_list

In [7]:
# rewrite the string into rows of of length m 
def friedman(ciphertext, m):                   
    # add padding to generate a matrix
    if len(ciphertext) % m !=0:
        padding = ' '
        s_len = len(ciphertext) + m - (len(ciphertext) %m)
    ciphertext=ciphertext.ljust(s_len, padding)
    #split the string into chuncks of length m
    matrix = [ciphertext[i:i+m] for i in range(0, len(ciphertext), m)]
    #matrix2 = list(map(''.join, zip(*[iter(ciphertext)]*m)))

    t_matrix = zip(*matrix)
  
    # rotate the cipher_dist by 1 for shift 1 to 25
    # store the product of two distribution in mgvalue  
    arr_ioc=[] #store ioc to verify the keyword length
    cipher_rows=[]  #store rows obtained from the same shift together
    for row in t_matrix:
        s= ''.join(row).rstrip(' ') #remove the paading to get the correct length
        ioc = idx_coincidence(s)
        arr_ioc.append(round(ioc, 3))
        cipher_rows.append(s)

    print("Index of Coindicence using key length 7:", arr_ioc)
    return cipher_rows

In [8]:
# Mutual index of coincidence per row per shift
def find_key(ciphertext):
    eng_dist = [.082,.015,.028,.043,.127,.022,.020,.061,.070,.002,.008,
               .040,.024,0.067,.075,0.19,.001,.060,.063,.091,.028,.010,.023,
               .001,.020,.001]
    cipher_rows = friedman(ciphertext, 7)

    mg_all = []
    for r in cipher_rows:
        cipher_dist = prob(r)
        
        # rotate the cipher_dist by 1 for shift 1 to 25
        # store the product of two distribution in mgvalue
        mgvalue = []

        for i in range(26):
            sum_product= 0

            for j in range(len(cipher_dist)):
                sum_product += eng_dist[j]*cipher_dist[j]
            cipher_dist = cipher_dist[1:] + cipher_dist[:1]
            mgvalue.append(round(sum_product, 3))
            
        mg_all.append(mgvalue)
        mgvalue = []
    return mg_all

In [9]:
# Use the mutual index of coincidence for each row to determine the key 
val = find_key(ciphertext)
print(val)

Index of Coindicence using key length 7: [0.063, 0.062, 0.07, 0.072, 0.069, 0.076, 0.074]
[[0.054, 0.039, 0.044, 0.047, 0.062, 0.042, 0.039, 0.043, 0.054, 0.038, 0.036, 0.051, 0.039, 0.044, 0.056, 0.067, 0.043, 0.039, 0.045, 0.061, 0.036, 0.039, 0.043, 0.034, 0.035, 0.041], [0.037, 0.053, 0.039, 0.039, 0.048, 0.038, 0.047, 0.052, 0.065, 0.04, 0.045, 0.04, 0.063, 0.041, 0.042, 0.043, 0.037, 0.04, 0.04, 0.055, 0.039, 0.051, 0.043, 0.062, 0.039, 0.036], [0.043, 0.043, 0.057, 0.041, 0.051, 0.043, 0.062, 0.037, 0.033, 0.035, 0.053, 0.041, 0.039, 0.05, 0.037, 0.047, 0.055, 0.071, 0.041, 0.041, 0.038, 0.058, 0.037, 0.039, 0.041, 0.036], [0.07, 0.041, 0.041, 0.039, 0.07, 0.043, 0.04, 0.041, 0.035, 0.041, 0.039, 0.062, 0.041, 0.045, 0.043, 0.07, 0.043, 0.039, 0.033, 0.056, 0.038, 0.034, 0.048, 0.034, 0.04, 0.045], [0.046, 0.034, 0.042, 0.035, 0.054, 0.039, 0.045, 0.041, 0.07, 0.043, 0.033, 0.045, 0.054, 0.041, 0.034, 0.051, 0.035, 0.046, 0.048, 0.067, 0.04, 0.039, 0.041, 0.066, 0.044, 0.037], [

In [10]:
# the highest Ic of each row is not unique, so the key is determined based on visual inspection
key = 'PIRATES'

# mapping
dictionary = list('ABCDEFGHIJKLMNOPQRSTUVWXYZ')

In [11]:
# rewrite the ciphertext as a table with 7 rows
cipher_rows=friedman(ciphertext, 7)

s=''
plaintext=[]
for i in range(len(cipher_rows)):
    for c in cipher_rows[i]:
        temp = (dictionary.index(c) - dictionary.index(key[i])) %26     
        s+=dictionary[temp]

    plaintext.append(s)
    #print(len(s), s)
    s=''

Index of Coindicence using key length 7: [0.063, 0.062, 0.07, 0.072, 0.069, 0.076, 0.074]


In [12]:
# prepare each line for matrix transformation
matrix2 =[]

for line in plaintext:
    if len(line) < len(plaintext[0]):
        padding=' '
        length = len(line) + len(plaintext[0]) - len(line)
        line = line.ljust(length, padding)
    matrix2.append(line.lower())
    
t_matrix2 = zip(*matrix2)

In [13]:
#concatenate the character and output to the screen
ptxt='' 
for row in t_matrix2:
    ptxt+=''.join(row)
print(ptxt)

psychiatristsarenotallowedtopracticepsychiatryiftheywereinsaneitisrequiredthattheyliterallydidnotknowwhattheyweretalkingabouttothisonecouldcounterthatyoudonothavetobeinfectedwithpneumoniainordeertoknowhowtocureitandyoudonothavetobeinfectedwithinsanitytoknowhowtocureiteitherbuttherebuttaltothatgoestothecoreofthewholeproblempneumoniaisabiologicalpatternitisscientificallyverifiableinsanityontheotherhandisanintellectualpatternitmayhavebiologicalcausesbutithasnophysicalorbiologicalrealitynoscientificinstrumentcanbeproducedincourttoshowwhoisinsaneandwhoissaneiftherewereonlyonepersonintheworldisthereanywayhecouldbeinsaneinsanityalwaysexistsinrelationtoothersitisasocialandintellectualdeviationnotabiologicaldeviationtheonlytestforinsanityinacourtoflaworanywhereelseisconformitytoaculturalstatusquothisbeingsoitfollowsthattheassignmentofmedicaldoctorstotreatinsanityisamisuseoftheirtrainingintellectualheresyisnotreallytheirbusinessmedicaldoctorsaretrainedtolookatthingsfromabiologicalperspective    

Plaintext: Psychiatrists are not allowed to practice psychiatry if they were insane. It is required that they literally did not know what they were talking about. To this, one could counter that you do not have to be infected with pneumonia in ordeer to know how to cure it and you do not have to be infected with insanity to know how to cure it either. But the rebuttal to that goes to the core of the whole problem. Pneumonia is a biological pattern. It is scientifically verifiable. Insanity on the other hand is an intellectual pattern. It may have biological causes but it has no physical or biological reality. No scientific instrument can be produced in court to show who is insane and who is sane. If there were only one person in the world, is there any way he could be insane? Insanity always exists in relation to others. It is a social and intellectual deviation, not a biological deviation. The only test for insanity in a court of law or anywhere else is conformity to a cultural status quo. This being so, it follows that the assignment of medical doctors to treat insanity is a misuse of their training. Intellectual heresy is not really their business. Medical doctors are trained to look at things from a biological perspective.